In [1]:
!pip install -q scikit-learn pyproj seaborn pandas seisbench pyproj obspy

In [8]:
!pip install pyocto

In [1]:
try:
    import obspy
    obspy.read()
except TypeError:
    # Needs to restart the runtime once, because obspy only works properly after restart.
    print('Stopping RUNTIME. If you run this code for the first time, this is expected. Colaboratory will restart automatically. Please run again.')
    exit()

In [2]:
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from pyproj import CRS, Transformer
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import torch
pd.set_option('future.no_silent_downcasting', True)
import seisbench.models as sbm

sns.set(font_scale=1.2)
sns.set_style("ticks")

In [3]:
# Projections
wgs84 = CRS.from_epsg(4326)
local_crs = CRS.from_epsg(9155)  # SIRGAS-Chile 2016 / UTM zone 19S
transformer = Transformer.from_crs(wgs84, local_crs)

In [4]:
client = Client("GFZ")

t0 = UTCDateTime("2014/05/01 00:00:00")
t1 = t0 + 12 * 60 * 60
# t1 = t0 + 24 * 60 * 60   # Full day, requires more memory
stream = client.get_waveforms(network="CX", station="*", location="*", channel="HH?", starttime=t0, endtime=t1)

inv = client.get_stations(network="CX", station="*", location="*", channel="HH?", starttime=t0, endtime=t1)

In [5]:
config={}
config["x(km)"] = (250, 600)
config["y(km)"] = (7200, 8000)
config["vel"] = {"P": 7.0, "S": 7.0 / 1.75} 
config["z(km)"] = (0, 250)
config["ncpu"] = 2
config['P_phase']=True
config['S_phase']=True
config['min_peak_pre_event']= 16
config['min_peak_pre_event_s']= 4
config['min_peak_pre_event_p']= 4
        

In [6]:
import seisbench
import torch
torch.set_num_threads(1)
seisbench.use_backup_repository()
picker = sbm.PhaseNet.from_pretrained("instance")



2024-11-16 00:24:44,224 | seisbench | WARNING | Setting remote root to: https://seisbench.gfz-potsdam.de/mirror/
Please note that this can affect your download speed.


/scicore/home/dokman0000/shi0000/anaconda3/envs/harpa/lib/python3.9/site-packages/seisbench/models/base.py:489: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_weights =

In [9]:
npicks_list=[]
harparesults_list=[]
import pyocto
for r in [1,0.5,0.1,0.05,0.01]:
    picks = picker.classify(stream, batch_size=256, P_threshold=0.075*r, S_threshold=0.1*r).picks
      # Output number of P and S picks
    npicks_list.append(len(picks))
    velocity_model = pyocto.VelocityModel0D(
    p_velocity=7.0,
    s_velocity=4.0,
    tolerance=2.0,
    association_cutoff_distance=250,
    )

    associator = pyocto.OctoAssociator.from_area(
        lat=(-25, -18),
        lon=(-71.5, -68),
        zlim=(0, 200),
        time_before=300,
        velocity_model=velocity_model,
        n_picks=10,
        n_p_and_s_picks=4,
    )
    stations = associator.inventory_to_df(inv)
    events, assignments = associator.associate_seisbench(picks, stations)
    associator.transform_events(events)
    harparesults_list.append(len(events))
    print(r,len(picks),len(events),Counter([p.phase for p in picks]))
    

The required number of P picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of P picks required will be 4.
The required number of S picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of S picks required will be 4.


1 7399 181 Counter({'P': 4903, 'S': 2496})


The required number of P picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of P picks required will be 4.
The required number of S picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of S picks required will be 4.


0.5 9349 210 Counter({'P': 6311, 'S': 3038})


The required number of P picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of P picks required will be 4.
The required number of S picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of S picks required will be 4.


0.1 15510 272 Counter({'P': 11024, 'S': 4486})


The required number of P picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of P picks required will be 4.
The required number of S picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of S picks required will be 4.


0.05 19065 300 Counter({'P': 13698, 'S': 5367})


The required number of P picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of P picks required will be 4.
The required number of S picks per event (3) is lower than the number of stations with both P and S pick (4). The effective number of S picks required will be 4.


0.01 31342 381 Counter({'P': 22221, 'S': 9121})


 20%|██        | 5/25 [00:32<02:03,  6.19s/it]

In [ ]:
# import pickle

# with open('pick_df.pkl', 'wb') as f:
#     pickle.dump(pick_df, f)
    
# with open('station_df.pkl', 'wb') as f:
#     pickle.dump(station_df, f)
# with open('config.pkl', 'wb') as f:
#     pickle.dump(config, f)
    
    
# import pickle

# # Load pick_df
# with open('pick_df.pkl', 'rb') as f:
#     pick_df = pickle.load(f)

# # Load station_df
# with open('station_df.pkl', 'rb') as f:
#     station_df = pickle.load(f)

# # Load config
# with open('config.pkl', 'rb') as f:
#     config = pickle.load(f)
